In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.metrics import accuracy_score, classification_report, f1_score
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn, sentiwordnet as swn
from token_embeddings import generate_embeddings
from tokenize_clean_text import clean_text

# Instantiate the WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()


Generate Embeddings

In [ ]:
#generate embeddings for our corpus, makes a file project_embeddings.csv and stores it in directory
generate_embeddings("only_tweet.csv")

In [ ]:
embeddings_df = pd.read_csv("project_embeddings.csv", index_col=0)

Clustering Function

In [ ]:
#clustering

def cluster_embeddings(df, num_of_clusters):
    
    kmeans = KMeans(n_clusters = num_of_clusters, random_state=42).fit(df)
    group_num = kmeans.labels_
    geo_centroids = kmeans.cluster_centers_
    
    #assign nearest word to geometric centroid in embedding space as centroid
    # find the index of the closest points from x to each class centroid
    close = pairwise_distances_argmin_min(geo_centroids, df, metric='euclidean')
    index_closest_points = close[0]
    word_centroids = df.iloc[index_closest_points].index
    
    #create dict of group number and centroids
    centroid_dict = {}
    for i in range(len(index_closest_points)):
        centroid_dict[i] = word_centroids[i]
    
    #create a dictionary of word and corresponding centroid
    
    #replace each label(group number) assigned by kmeans cluster algo with centroid word
    cen = [centroid_dict.get(group) for group in group_num]

    #create a dictionary
    word_centroid_dict = {}

    for i in range(df.shape[0]):
        word_centroid_dict[df.index[i]] = cen[i]
    
    return word_centroid_dict
    

Replace words in reviews with their cluster centroids and then calculate score

In [ ]:
def replace_with_centroids(review, word_centroid_dict):
    
    new_review = [word_centroid_dict.get(word) if word in word_centroid_dict else word for word in review]
    
    return new_review    
    
    

def swn_classifier(review):
 
    sentiment = 0.0
    tokens_count = 0.0
    
    #Calculating score
    for word in review:
        
        lemma = lemmatizer.lemmatize(word)
        if not lemma:
            continue
 
        synsets = wn.synsets(lemma)
        if not synsets:
            continue
        
        # Take the first synset, the most common
        synset = synsets[0]
        swn_synset = swn.senti_synset(synset.name())
        print()
 
        #sentiment is the difference between positive and negative score
        sentiment += swn_synset.pos_score() - swn_synset.neg_score()
        tokens_count += 1
 
    # Default: neither positive, nor negative
    if not tokens_count:
        return 0
 
    return sentiment

Calling the cluster function to generate a word centroid dictionary. Num is a hyperparameter to be adjusted for best accuracy

In [ ]:
#calling the cluster function to generate a word centroid dictionary
num = 2000   #roughly 11 words per cluster
word_centroid_dict = cluster_embeddings(embeddings_df, num)
    

Import data file, process it(sentiment classification) and export it in required format

In [ ]:
#import reviews file
#use both review title and review content columns to predict score
reviews = pd.read_csv("only_tweet.csv")

#arrays to store list of tokens, replaced words, scores
clean_tokens = []
replaced_tokens = []
y_predicted = []


#calling the main calculate function
for review in reviews["text"]:    
    clean_t = clean_text(review)
    clean_tokens.append(clean_t)
    
    replaced_t = replace_with_centroids(clean_t, word_centroid_dict)
    replaced_tokens.append(replaced_t)
    
    senti_score = swn_classifier(replaced_t)
    y_predicted.append(senti_score)

    
#Classify reviews according to setiment score assigned
#1 : positive, 0 : neutral, -1 : negative 
y_classified = []
for i in y_predicted:
    if i > 0:
        y_classified.append(1)
    elif i<0:
        y_classified.append(-1)
    elif i==0:
        y_classified.append(0)
        

#appending cols in df
reviews["tokens"] = clean_tokens
reviews["replaced_centroids"] = replaced_tokens
reviews["sentiment_score"] = y_predicted
reviews["predicted_sentiment"] = y_classified

#exporting df
reviews.to_csv("classified_full_review_embeddings.csv", header=True)


In [ ]:
reviews.head()

In [ ]:
reviews.to_csv("op_embeddings_score.csv", header=True)

In [ ]:
op_word_embed.head()

In [ ]:
accuracy_score(reviews["true_sentiment"], y_classified)

In [ ]:
f1_score(reviews["true_sentiment"], y_classified, average=None)

In [ ]:
#Some examples:
for n in range(0,100,20):
    print("user_rating:",reviews["rating"][n] )
    print("review:",reviews["full_review"][n])
    print("tokens:",reviews["tokens"][n])
    print("replaced centroids:",reviews["replaced_centroids"][n])
    print("sentiment_score:",reviews["sentiment_score"][n],'\n')